In [1]:
import pandas as pd
# import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

rstudio on gphost01
http://gphost01.bcgsc.ca:8787/

local rstudio to make lolliplot: use this R:
export RSTUDIO_WHICH_R=/gsc/software/linux-x86_64-centos6/R-3.3.2/bin/R

In [62]:
wkdir = '/projects/da_workspace/software/mutation_rainstorm/118_patients_noncoding/'
inmaf = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients.maf'
valhotspotsf = '/projects/trans_scratch/validations/workspace/szong/Cervical/hotspots/123_patients/sliding_window_mutaional_distance_merged_overlap_promoter_txn_factor_merged.txt'
noncoding_maf = f'{wkdir}118_patients_noncoding.maf'
fname = 'waveletPatientDetail_withMaf.tsv'
infile = f'{wkdir}{fname}'
outfile = f'{infile}.parsed'
outfile

'/projects/da_workspace/software/mutation_rainstorm/118_patients_noncoding/waveletPatientDetail_withMaf.tsv.parsed'

# 1. overlap rainstorm hotspots with 842 hotspots in validation

In [61]:
valhotspots = pd.read_csv(valhotspotsf, sep='\t', header=None)
valhotspots[:2]

,0,1,2
0,1,10089,10578
1,1,16132,16352


# 1. some overview of non-coding hotspot landscape

In [50]:
df = pd.read_csv(infile, sep='\t', low_memory=False)
df.head()

,chromosome,leftPosition,rightPosition,peakPosition,peakMinValue,numberPatients,mutPerKbMaf,mostFreqVClassification,position,patient,...,ExAC_AF,ExAC_AF_AFR,ExAC_AF_AMR,ExAC_AF_EAS,ExAC_AF_FIN,ExAC_AF_NFE,ExAC_AF_OTH,ExAC_AF_SAS,GENE_PHENO,FILTER
1_130.1,10,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118649,HTMCP.03.06.02075_T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS
1_130.2,10,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118649,HTMCP.03.06.02340_T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS
1_130.3,10,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118654,HTMCP.03.06.02075_T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS
1_130.4,10,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118680,HTMCP.03.06.02075_T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS
1_130.5,10,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118680,HTMCP.03.06.02242_T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS


In [51]:
df.shape
df['hotspot_length'] = df['rightPosition'] - df['leftPosition']

(55560, 124)

In [54]:
df[df.End_Position.isna()].shape
# be aware some positions are invalid, rainstorm issue?

(67, 125)

In [44]:
keep = ['Chromosome','End_Position', 'chromosome', 'leftPosition', 'rightPosition', 'peakPosition',
       'peakMinValue', 'numberPatients', 'mutPerKbMaf',
       'mostFreqVClassification', 'position', 'patient', 'hotspot_length', 'Hugo_Symbol']

In [33]:
df.Variant_Classification.unique()
df.Hugo_Symbol.unique().shape

array(['Intron', "5'Flank", "3'Flank", 'IGR', "5'UTR", 'RNA', "3'UTR",
       'Splice_Site', nan], dtype=object)

(1726,)

In [55]:
# df[keep].sort_values('hotspot_length')
dff = df[keep]
dff = dff[dff.End_Position.notnull()]
dff['End_Position'] = dff['End_Position'].astype(int) +1
dff['Start_Position'] = dff['End_Position'] -1
dff.head(2)

,Chromosome,End_Position,chromosome,leftPosition,rightPosition,peakPosition,peakMinValue,numberPatients,mutPerKbMaf,mostFreqVClassification,position,patient,hotspot_length,Hugo_Symbol,Start_Position
1_130.1,10,118650,10,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118649,HTMCP.03.06.02075_T,9298,TUBB8,118649
1_130.2,10,118650,10,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118649,HTMCP.03.06.02340_T,9298,TUBB8,118649


In [59]:
keep = ['Chromosome','Start_Position', 'End_Position', 'leftPosition', 'rightPosition', 'peakPosition',
       'peakMinValue', 'numberPatients', 'mutPerKbMaf',
       'mostFreqVClassification', 'position', 'patient', 'hotspot_length', 'Hugo_Symbol']
dff = dff[keep]
dff.head()
dff.to_csv(f'{wkdir}all_muts.bed', index=False, sep='\t', header=False)

,Chromosome,Start_Position,End_Position,leftPosition,rightPosition,peakPosition,peakMinValue,numberPatients,mutPerKbMaf,mostFreqVClassification,position,patient,hotspot_length,Hugo_Symbol
1_130.1,10,118649,118650,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118649,HTMCP.03.06.02075_T,9298,TUBB8
1_130.2,10,118649,118650,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118649,HTMCP.03.06.02340_T,9298,TUBB8
1_130.3,10,118654,118655,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118654,HTMCP.03.06.02075_T,9298,TUBB8
1_130.4,10,118680,118681,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118680,HTMCP.03.06.02075_T,9298,TUBB8
1_130.5,10,118680,118681,118649,127947,124617,-4.725319,34,6.774922,3'Flank,118680,HTMCP.03.06.02242_T,9298,TUBB8


In [27]:
dff[dff.mostFreqVClassification=='IGR'].sort_values(['hotspot_length', 'numberPatients'], ascending=[True, False])

,chromosome,leftPosition,rightPosition,peakPosition,peakMinValue,numberPatients,mutPerKbMaf,mostFreqVClassification,position,patient,hotspot_length,Hugo_Symbol
1_57317.1,X,82973150,82973150,82973150,-7.547693,9,9000.000000,IGR,82973150,HTMCP.03.06.02074_T,0,UnknownGene
1_57317.2,X,82973150,82973150,82973150,-7.547693,9,9000.000000,IGR,82973150,HTMCP.03.06.02092_T,0,UnknownGene
1_57317.3,X,82973150,82973150,82973150,-7.547693,9,9000.000000,IGR,82973150,HTMCP.03.06.02197_T,0,UnknownGene
1_57317.4,X,82973150,82973150,82973150,-7.547693,9,9000.000000,IGR,82973150,HTMCP.03.06.02209_T,0,UnknownGene
1_57317.5,X,82973150,82973150,82973150,-7.547693,9,9000.000000,IGR,82973150,HTMCP.03.06.02259_T,0,UnknownGene
1_57317.6,X,82973150,82973150,82973150,-7.547693,9,9000.000000,IGR,82973150,HTMCP.03.06.02320_T,0,UnknownGene
1_57317.7,X,82973150,82973150,82973150,-7.547693,9,9000.000000,IGR,82973150,HTMCP.03.06.02392_T,0,UnknownGene
1_57317.8,X,82973150,82973150,82973150,-7.547693,9,9000.000000,IGR,82973150,HTMCP.03.06.02435_T,0,UnknownGene
1_57317.9,X,82973150,82973150,82973150,-7.547693,9,9000.000000,IGR,82973150,HTMCP.03.06.02437_T,0,UnknownGene
1_18886.1,10,38804327,38804327,38804327,-6.852316,7,7000.000000,IGR,38804327,HTMCP.03.06.02174_T,0,UnknownGene


# 2. look at promoter or nearby enhancers

In [41]:
dff[(dff.Hugo_Symbol.notnull())&(dff.mostFreqVClassification !='IGR')&(dff.mostFreqVClassification !='Intron')].sort_values(['hotspot_length', 'numberPatients'], ascending=[True, False])

,chromosome,leftPosition,rightPosition,peakPosition,peakMinValue,numberPatients,mutPerKbMaf,mostFreqVClassification,position,patient,hotspot_length,Hugo_Symbol
1_18041.1,5,20578062,20578062,20578062,-7.596462,7,7000.000000,5'Flank,20578062,HTMCP.03.06.02040_T,0,CDH18
1_18041.2,5,20578062,20578062,20578062,-7.596462,7,7000.000000,5'Flank,20578062,HTMCP.03.06.02103_T,0,CDH18
1_18041.3,5,20578062,20578062,20578062,-7.596462,7,7000.000000,5'Flank,20578062,HTMCP.03.06.02185_T,0,CDH18
1_18041.4,5,20578062,20578062,20578062,-7.596462,7,7000.000000,5'Flank,20578062,HTMCP.03.06.02194_T,0,CDH18
1_18041.5,5,20578062,20578062,20578062,-7.596462,7,7000.000000,5'Flank,20578062,HTMCP.03.06.02195_T,0,CDH18
1_18041.6,5,20578062,20578062,20578062,-7.596462,7,7000.000000,5'Flank,20578062,HTMCP.03.06.02202_T,0,CDH18
1_18041.7,5,20578062,20578062,20578062,-7.596462,7,7000.000000,5'Flank,20578062,HTMCP.03.06.02266_T,0,CDH18
1_1281.1,13,19067704,19067704,19067704,-7.159624,5,5000.000000,3'Flank,19067704,HTMCP.03.06.02020_T,0,BNIP3P7
1_1281.2,13,19067704,19067704,19067704,-7.159624,5,5000.000000,3'Flank,19067704,HTMCP.03.06.02074_T,0,BNIP3P7
1_1281.3,13,19067704,19067704,19067704,-7.159624,5,5000.000000,3'Flank,19067704,HTMCP.03.06.02261_T,0,BNIP3P7


In [9]:
df.columns

Index(['chromosome', 'leftPosition', 'rightPosition', 'peakPosition',
       'peakMinValue', 'numberPatients', 'mutPerKbMaf',
       'mostFreqVClassification', 'position', 'patient',
       ...
       'ExAC_AF_AFR', 'ExAC_AF_AMR', 'ExAC_AF_EAS', 'ExAC_AF_FIN',
       'ExAC_AF_NFE', 'ExAC_AF_OTH', 'ExAC_AF_SAS', 'GENE_PHENO', 'FILTER',
       'hotspot_length'],
      dtype='object', length=125)

In [11]:
df1 = df[['Chromosome', 'position',  'chromosome', 'leftPosition', 'rightPosition', 'mostFreqVClassification', 'patient', 'Gene', 'SYMBOL']]

In [12]:
df2 = df1[df1.SYMBOL=='TERT']

In [13]:
df2.head()
df2.tail()

,Chromosome,position,chromosome,leftPosition,rightPosition,mostFreqVClassification,patient,Gene,SYMBOL
1_12161.1,5,1283266,5,1283266,1284668,Intron,HTMCP.03.06.02194_T,ENSG00000164362,TERT
1_12161.2,5,1283567,5,1283266,1284668,Intron,HTMCP.03.06.02058_T,ENSG00000164362,TERT
1_12161.3,5,1283582,5,1283266,1284668,Intron,HTMCP.03.06.02058_T,ENSG00000164362,TERT
1_12161.4,5,1283655,5,1283266,1284668,Intron,HTMCP.03.06.02268_T,ENSG00000164362,TERT
1_12161.5,5,1283705,5,1283266,1284668,Intron,HTMCP.03.06.02076_T,ENSG00000164362,TERT


,Chromosome,position,chromosome,leftPosition,rightPosition,mostFreqVClassification,patient,Gene,SYMBOL
1_1259.9,5,1295228,5,1295228,1295250,5'Flank,HTMCP.03.06.02346_T,ENSG00000164362,TERT
1_1259.10,5,1295228,5,1295228,1295250,5'Flank,HTMCP.03.06.02377_T,ENSG00000164362,TERT
1_1259.11,5,1295250,5,1295228,1295250,5'Flank,HTMCP.03.06.02148_T,ENSG00000164362,TERT
1_1259.12,5,1295250,5,1295228,1295250,5'Flank,HTMCP.03.06.02176_T,ENSG00000164362,TERT
1_1259.13,5,1295250,5,1295228,1295250,5'Flank,HTMCP.03.06.02259_T,ENSG00000164362,TERT


In [21]:
df2['new_pat'] = df2['patient'].apply(lambda x: '-'.join(x.split('_')[0].split('.')))

/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
posf = '/projects/trans_scratch/validations/workspace/szong/Cervical/73_hiv_positive_patients.txt'
negf = '/projects/trans_scratch/validations/workspace/szong/Cervical/45_hiv_negative_patients.txt'
pos = pd.read_csv(posf, header=None)[0].tolist()
pos[:2]
len(pos)
neg = pd.read_csv(negf, header=None)[0].tolist()
neg[:2]
len(neg)

['HTMCP-03-06-02001', 'HTMCP-03-06-02003']

73

['HTMCP-03-06-02002', 'HTMCP-03-06-02006']

45

In [42]:
df2.shape
df2[df2.new_pat.isin(pos)&(df2.leftPosition==1283266)]['new_pat'].nunique()
df2[df2.new_pat.isin(neg)]

(36, 10)

11

,Chromosome,position,chromosome,leftPosition,rightPosition,mostFreqVClassification,patient,Gene,SYMBOL,new_pat
1_12161.1,5,1283266,5,1283266,1284668,Intron,HTMCP.03.06.02194_T,ENSG00000164362,TERT,HTMCP-03-06-02194
1_12161.7,5,1283897,5,1283266,1284668,Intron,HTMCP.03.06.02256_T,ENSG00000164362,TERT,HTMCP-03-06-02256
1_12161.8,5,1283897,5,1283266,1284668,Intron,HTMCP.03.06.02326_T,ENSG00000164362,TERT,HTMCP-03-06-02326
1_12161.9,5,1284161,5,1283266,1284668,Intron,HTMCP.03.06.02006_T,ENSG00000164362,TERT,HTMCP-03-06-02006
1_12161.10,5,1284168,5,1283266,1284668,Intron,HTMCP.03.06.02322_T,ENSG00000164362,TERT,HTMCP-03-06-02322
1_12161.19,5,1284652,5,1283266,1284668,Intron,HTMCP.03.06.02063_T,ENSG00000164362,TERT,HTMCP-03-06-02063
1_12161.23,5,1284668,5,1283266,1284668,Intron,HTMCP.03.06.02328_T,ENSG00000164362,TERT,HTMCP-03-06-02328
1_1259.1,5,1295228,5,1295228,1295250,5'Flank,HTMCP.03.06.02012_T,ENSG00000164362,TERT,HTMCP-03-06-02012
1_1259.6,5,1295228,5,1295228,1295250,5'Flank,HTMCP.03.06.02326_T,ENSG00000164362,TERT,HTMCP-03-06-02326
1_1259.7,5,1295228,5,1295228,1295250,5'Flank,HTMCP.03.06.02328_T,ENSG00000164362,TERT,HTMCP-03-06-02328


In [99]:
df2['chr'] = 'chr'+ df2.Chromosome
df2['end'] = df2.position + 1

/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [100]:
df2[['chr', 'position', 'end']]

,chr,position,end
1_95223.1,chr3,178936082,178936083
1_95223.2,chr3,178936082,178936083
1_95223.3,chr3,178936082,178936083
1_95223.4,chr3,178936082,178936083
1_95223.5,chr3,178936082,178936083
1_95223.6,chr3,178936082,178936083
1_95223.7,chr3,178936082,178936083
1_95223.8,chr3,178936082,178936083
1_95223.9,chr3,178936091,178936092
1_95223.10,chr3,178936091,178936092


In [101]:
of = f'{wkdir}PIK3CA.bed'
df2[['chr', 'position', 'end']].to_csv(of, index=False, sep='\t' )

In [19]:
df1[df1.chromosome.isnull()]

,chromosome,leftPosition,rightPosition,mostFreqVClassification,patient


In [48]:
df1['SYMBOL'] = df1.SYMBOL.astype(str)

/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
df1.groupby(['chromosome', 'leftPosition', 'rightPosition']).agg({'SYMBOL':lambda x: ','.join(set(x))})

SYMBOL
chromosome leftPosition rightPosition                           
1          10235        17038                            DDX11L1
           533206       536216         RP5-857K21.3,RP5-857K21.2
           725020       727962                     RP11-206L10.9
           755226       758168                  FAM87B,LINC01128
           830098       832212                               nan
           1034108      1035272                         C1orf159
           1111306      1111716                           TTLL10
           1316309      1318167                CCNL2,RP5-890O3.3
           1541009      1541531                              nan
           1647901      1650472                           CDK11A
           2193258      2193730                              SKI
           2585854      2589655                            TTC34
           2608816      2609171                            TTC34
           2617146      2618218                            TTC34
           2627812      2631787                            TTC34
           2688037      2690083                            TTC34
           2765522      2766132                              nan
           2891195      2891214                              nan
           3097664      3098381                           PRDM16
           4136892      4138678                              nan
           4203107      4204726                              nan
           4391955      4392174                              nan
           4691703      4693297                              nan
           4724618      4726826                            AJAP1
           5220251      5220378                              nan
           5727629      5735888                RP11-154H17.1,nan
           5875842      5877084                       AL356693.1
           7050863      7050865                           CAMTA1
           7342433      7344237                           CAMTA1
           8404850      8404851                          SLC45A1
...                                                          ...
X          136205070    136205818                            nan
           137527452    137528284                            nan
           138424252    138424382                            nan
           139805530    139807421                      LINC00632
           140877287    140878550                            nan
           140992501    140994490                         MAGEC1
           141377240    141377776                            nan
           141637709    141638785                            nan
           141656560    141656894                            nan
           141932218    141933124                            nan
           142387895    142389670                   GS1-256O22.5
           143930710    143932111                            nan
           144575858    144576668                            nan
           144836749    144837505                            nan
           144999423    144999581                            nan
           146132214    146133862                            nan
           146206285    146207149                            nan
           146380092    146380228                            nan
           146442089    146443984                            nan
           146643777    146644512                            nan
           147061629    147062754                         FMR1NB
           147276524    147277663                       U40455.1
           147662541    147663137                           AFF2
           149471253    149472572                            nan
           150082609    150083551                     AF003626.1
           150240227    150241450                            nan
           151443147    151443934                         GABRA3
           153013061    153014728                          ABCD1
           154797878    154798634                          TMLHE
           155251445    1552

In [68]:
# aggregate on multiple columns
d = {'patient': ['nunique'], 'SYMBOL':lambda x: ','.join(set(x))}
df2 = df1.groupby(['chromosome', 'leftPosition', 'rightPosition']).agg(d).reset_index()
df2.head(2)
df2.columns = df2.columns.get_level_values(0)
df2.head(2)
df3 = df2.sort_values('patient', ascending=False)

,chromosome,leftPosition,rightPosition,patient,SYMBOL
,,,,nunique,<lambda>
0,1,10235,17038,37,DDX11L1
1,1,533206,536216,21,"RP5-857K21.3,RP5-857K21.2"


,chromosome,leftPosition,rightPosition,patient,SYMBOL
0,1,10235,17038,37,DDX11L1
1,1,533206,536216,21,"RP5-857K21.3,RP5-857K21.2"


In [70]:
df3.to_csv(outfile, sep='\t', index=False)

In [34]:
df1.groupby(['chromosome', 'leftPosition', 'rightPosition'])['patient', 'SYMBOL'].nunique().sort_values('patient', ascending=False).reset_index().head(30)

,chromosome,leftPosition,rightPosition,patient,SYMBOL
0,1,16829943,16840992,79,3
1,10,127574661,127589465,75,3
2,16,33037154,33041167,74,1
3,16,33063091,33066469,73,1
4,7,152073997,152087464,73,1
5,9,33792343,33799648,72,1
6,5,49432029,49439927,69,0
7,15,22822737,22832411,66,1
8,3,195230629,195232990,65,0
9,1,149025692,149032010,65,1


In [12]:
df1.groupby(['chromosome', 'leftPosition', 'rightPosition'])['patient'].nunique().sort_values(ascending=False).head(30)

chromosome  leftPosition  rightPosition
1           16829943      16840992         79
10          127574661     127589465        75
16          33037154      33041167         74
            33063091      33066469         73
7           152073997     152087464        73
9           33792343      33799648         72
5           49432029      49439927         69
15          22822737      22832411         66
1           149025692     149032010        65
X           88457388      88462267         65
3           195230629     195232990        65
2           92281655      92283918         63
7           65053792      65060659         63
4           49153176      49154983         62
15          20136470      20138745         62
4           3557457       3571431          62
9           45357066      45366189         61
10          42638043      42640403         60
X           155239        162703           60
7           142458277     142469657        59
            64960477      64962979      